[View in Colaboratory](https://colab.research.google.com/github/manan-arya/Analytics_Vidhya_Competition/blob/XGBoost/XGBoost_CV.ipynb)

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV



In [77]:
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv

--2018-09-02 10:22:26--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12564851 (12M) [text/plain]
Saving to: ‘train_NIR5Yl1.csv.2’

train_NIR5Yl1.csv.2 100%[===================>]  11.98M  21.4MB/s    in 0.6s    

2018-09-02 10:22:27 (21.4 MB/s) - ‘train_NIR5Yl1.csv.2’ saved [12564851/12564851]

--2018-09-02 10:22:28--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting resp

In [0]:
train=pd.read_csv("train_NIR5Yl1.csv")
test=pd.read_csv("test_8i3B3FC.csv")

In [79]:
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [80]:
test.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [81]:
train = pd.concat([train,pd.get_dummies(train['Tag'], drop_first=True)], axis=1)
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [82]:
test = pd.concat([test,pd.get_dummies(test['Tag'], drop_first=True)], axis=1)
test.head()

,ID,Tag,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,366953,a,5645.0,3.0,50652,33200.0,0,0,0,0,0,0,0,0,0
1,71864,c,24511.0,6.0,37685,2730.0,1,0,0,0,0,0,0,0,0
2,141692,i,927.0,1.0,135293,21167.0,0,0,1,0,0,0,0,0,0
3,316833,i,21.0,6.0,166998,18528.0,0,0,1,0,0,0,0,0,0
4,440445,i,4475.0,10.0,53504,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
train.drop("Tag",axis=1,inplace=True)
test.drop("Tag",axis=1,inplace=True)
ID=test['ID']


In [0]:
train.drop("ID",axis=1,inplace=True)
test.drop("ID",axis=1,inplace=True)

In [85]:
train.head()

,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
X=train.drop("Upvotes",axis=1)
y=train["Upvotes"]


In [0]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, random_state=123)

In [88]:
DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg = xgb.train(params = params, dtrain=DM_train,num_boost_round=200)

preds = xg_reg.predict(DM_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))



RMSE: 2742.814975


In [37]:
review_data_matrix=xgb.DMatrix(data=X,label=y)

untuned_params = {"objective":"reg:linear"}

tuned_cv_rmse = xgb.cv(dtrain = review_data_matrix,params=untuned_params, nfold=7, num_boost_round=200, metrics="rmse",as_pandas=True, seed=123)

print(type(tuned_cv_rmse))

print("Tuned rmse: %f" %((tuned_cv_rmse["test-rmse-mean"]).tail(1)))


KeyboardInterrupt: ignored

In [21]:
new_preds=tuned_cv_rmse.predict(X_test)
new_rmse = np.sqrt(mean_squared_error(y_test,new_preds))
print("RMSE: %f" % (new_rmse))


AttributeError: ignored

#Tuning XGB

In [89]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    #'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    #'eval_metric': 'mae',
    #'lambda': 0.8,   
    #'alpha': 0.4, 
    #'silent': 1
}

# AttributeError: module 'xgboost' has no attribute 'DMatrix'
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

num_boost_rounds = 150
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, 
num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
xg_preds = model.predict(dtest)

rmse = np.sqrt(mean_squared_error(y_test,xg_preds))
print("RMSE: %f" % (rmse))


Setting up data for XGBoost ...
num_boost_rounds=150

Training XGBoost ...

Predicting with XGBoost ...
RMSE: 1459.252458


In [90]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    #'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    #'eval_metric': 'mae',
    #'lambda': 0.8,   
    #'alpha': 0.4, 
    #'silent': 1
}

# AttributeError: module 'xgboost' has no attribute 'DMatrix'
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(test)

num_boost_rounds = 150
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, 
num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
preds = model.predict(dtest)


Setting up data for XGBoost ...
num_boost_rounds=150

Training XGBoost ...

Predicting with XGBoost ...


In [0]:


submission = pd.DataFrame({'ID':ID, 'Upvotes':preds})


In [0]:
submission.to_csv('enigma_sub_3.csv', index=False)
from google.colab import files 
files.download('enigma_sub_3.csv')

# Grid Search CV on split


In [0]:
n_estimator = [10,20,30,40,50]

params = {
    "objective" : "reg:linear",
    "max_depth" :  5
}


best_rmse = []
    
datamatrix = xgb.DMatrix(data=X_train, label=y_train)



for depth in n_estimator:
    clf = xgb.XGBRegressor()
    params["n_estimators"] = depth
    cv_results = xgb.cv( dtrain=datamatrix, params = params, folds=4, num_boost_round = 10, metrics="rmse", seed=123, as_pandas=True)
    print(cv_results)
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])
    
#     clf.fit(final_data, final_label)

#print(pd.DataFrame(list(zip(n_estimator, best_rmse)), columns=["n_estimator","best_rmse"]))

In [92]:
clf.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
pred_cv=clf.predict(X_test)

In [94]:
rmse_cv = np.sqrt(mean_squared_error(y_test,pred_cv))
print("RMSE: %f" % (rmse_cv))

RMSE: 1312.345934


# Grid Search On actual

In [0]:
n_estimator = [10,20,30,40,50]

params = {
    "objective" : "reg:linear",
    "max_depth" :  5
}


best_rmse = []
    
datamatrix = xgb.DMatrix(data=X, label=y)



for depth in n_estimator:
    clf = xgb.XGBRegressor()
    params["n_estimators"] = depth
    cv_results = xgb.cv( dtrain=datamatrix, params = params, folds=7, num_boost_round = 20, metrics="rmse", seed=123, as_pandas=True)
    #print(cv_results)
    #best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])
    
clf.fit(X,y)

#print(pd.DataFrame(list(zip(n_estimator, best_rmse)), columns=["n_estimator","best_rmse"]))

predictions=clf.predict(test)

In [0]:
submission_gscv = pd.DataFrame({'ID':ID, 'Upvotes':predictions})

In [0]:
submission_gscv.to_csv('enigma_sub_6.csv', index=False)
from google.colab import files 
files.download('enigma_sub_6.csv')

# Grid Search again

In [102]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03], #so called `eta` value
              'max_depth': [5],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [50]}




xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Traceback (most recent call last):
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = g

KeyboardInterrupt: ignored